In [ ]:
# identify non-English language postings, as well as any postings that contain emojis (suspected advertisements)
for i in df_all_subm['title']:
    if i.isascii() == False:
        print(i)

In [ ]:
# false positive
df_all_subm.iloc[6]['title']

In [ ]:
for i in df_all_subm.iloc[6]['title']:
    if i.isascii() == False:
        print(i)

In [ ]:
# replace non-ascii apostrophe with ascii apostrophe
df_all_subm['title'] = df_all_subm['title'].replace("’", "'", regex = True)

In [ ]:
# identify non-English language postings, as well as any postings that contain emojis (suspected advertisements)
for i in df_all_subm['title']:
    if i.isascii() == False:
        print(i)

In [ ]:
# still reaching a large volume of false positives

# change approach, replace all non-alphanumeric/digit characters with underscore _

for i in range(len(df_all_subm['title'])):
    object_1 = df_all_subm['title']
    for char in range(len(object_1[i])):
        if object_1[i][char].isascii() == False:
            object_2 = object_1[i][char]
            object_1[i] = object_1[i].replace(object_2, '_')

In [ ]:
# find ratio of underscore:len(string) for all strings in column
# test various threshhold ratios

In [ ]:
for i in df_all_subm['title']:
    measure = len(i)
    count = 0
    for char in i:
        if char == '_':
            count += 1
    if count/measure >= 0.06:
        print(i)

In [ ]:
# function to drop rows below non-ascii:len(string) ratio threshold of 4.5%

def non_ascii_dropper(dataframe, column, threshold_ratio):
    for i in range(len(dataframe[column])):
        for char in range(len(object_1[i])):
            if dataframe[column][i][char].isascii() == False:
                non_ascii = dataframe[column][i][char]
                dataframe[column][i] = dataframe[column][i].replace(non_ascii, '_')
    for i in range(len(dataframe[column])):
        measure = len(dataframe[column][i])
        count = 0
        for char in dataframe[column][i]:
            if char == '_':
                count += 1
        if count/measure >= 0.045:
            dataframe.drop([i], axis = 0, inplace = True)

In [ ]:
non_ascii_dropper(df_all_subm, 'title', 0.045)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
rfc = RandomForestClassifier()

rfc_pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('rfc', RandomForestClassifier())
])


rfc_param = {
    'cvec__ngram_range': [(1,2), (1,3)],
    'cvec__stop_words': [None, stop_list],
    'rfc__n_estimators': [400, 500, 600]
}

gs_rfc = GridSearchCV(rfc_pipe, param_grid = rfc_param, n_jobs = -1)

gs_rfc.fit(X_train, y_train)

gs_rfc.best_params_

In [ ]:
gs_rfc.score(X_train, y_train), gs_rfc.score(X_test, y_test)

In [ ]:
gbc_pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('gbc', GradientBoostingClassifier())
])

gbc_param = {
    'cvec__ngram_range': [(1,1), (1,2),(2,2),(1, 3),(2,3), (3,3)],
    'cvec__tokenizer': [my_lemma, None],
    'gbc__n_estimators' : [200, 300, 400, 500, 600]
}

gs_gbc = GridSearchCV(gbc_pipe, param_grid = gbc_param, n_jobs = -1)

gs_gbc.fit(X_train, y_train)

gs_gbc.best_params_

In [ ]:
gs_gbc.score(X_train, y_train), gs_gbc.score(X_test, y_test), gs_gbc.score(X_final, y_final)